In [1]:
import os, sys
import glob
import numpy as np
sys.path.append(os.path.join(os.getenv('LAMBO_HOME'), 'lambo/scripts/'))
from astropy.table import Table, vstack, hstack

from hsc_gaap.compile_catalogs import compileCatalogs


Overriding default configuration file with /projects/HSC/LSST/stack_20220527/conda/envs/lsst-scipipe-4.1.0/share/eups/Linux64/dustmaps_cachedata/g41a3ec361e+62d12e78cb/config/.dustmapsrc


In [155]:
# jiaxuan_table = Table.read("/projects/MERIAN/repo/S20A/gaapTable/9813/objectTable_S20A_Merian_9813_221224.fits")
# gaap_table  = Table.read('/scratch/gpfs/am2907/Merian/gaap/S20A/gaapTable/8280/1,8/objectTable_8280_1,8_S20A.fits')

# keepColumns = jiaxuan_table.colnames[:5]
# keepColumns += jiaxuan_table.colnames[83:]

# keepColumns += ["objectId"]
# keepColumns = np.array(keepColumns)
# np.save("keep_table_columns", keepColumns)

In [156]:
# merian_cols = consolidateMerianCats([0], 9813).colnames
# gaap_cols = Table.read('/scratch/gpfs/am2907/Merian/gaap/S20A/gaapTable/9318/0,0/objectTable_9318_0,0_S20A.fits').colnames  
# keepColumns_merian = [col for col in merian_cols if (np.sum([k in col for k in keepColumns])>0)]
# keepColumns_gaap   = [col for col in gaap_cols if (np.sum([k in col for k in keepColumns])>0)]

In [128]:
# keepColumns = list(np.load("keep_table_columns.npy"))

In [129]:
keepColumns = open("keep_table_columns.txt", "r").read().split('\n')
merian_cols = consolidateMerianCats([0], 9813).colnames
gaap_cols = Table.read('/scratch/gpfs/am2907/Merian/gaap/S20A/gaapTable/9318/0,0/objectTable_9318_0,0_S20A.fits').colnames  
keepColumns_merian = np.array([[f"{filt}_"+ col for col in keepColumns[6:-1]] for filt in ['N708', 'N540']]).flatten()
keepColumns_merian = np.concatenate((keepColumns[:6], keepColumns_merian))
keepColumns_gaap = np.array([[f"{filt}_"+ col for col in keepColumns[6:]] for filt in ['g', 'r', 'i', 'z', 'y']]).flatten()
keepColumns_gaap = np.concatenate((keepColumns[:6], keepColumns_gaap))
keepColumns_merian = [k for k in keepColumns_merian if k in merian_cols] 
keepColumns_gaap   = [k for k in keepColumns_gaap if k in gaap_cols] 

In [152]:
np.save("keep_table_columns_merian", keepColumns_merian)
np.save("keep_table_columns_gaap", keepColumns_gaap)

---
Try fewer columns

In [2]:
compileCatalogs([8280])

COMPILED TABLE OF 69666 ROWS and 69 COLUMNS
WROTE TABLE TO /scratch/gpfs/am2907/Merian/gaap/S20A/gaapTable/8280/objectTable_8280_S20A.fits


In [145]:
tract = 9326

repo = "/scratch/gpfs/am2907/Merian/gaap/"

patches = findReducedPatches(tract)
patches.sort()
catDir = os.path.join(repo, f"S20A/gaapTable/{tract}/")
objectTables = np.array(glob.glob(catDir + "*/objectTable*"))

ot_patches = np.array([ot.split("_")[2] for ot in objectTables])
ot_patches = np.array([int(otp[0]) + int(otp[2])*9 for otp in ot_patches])
objectTables, ot_patches = objectTables[ot_patches.argsort()], ot_patches[ot_patches.argsort()]

assert np.all(patches == ot_patches), "Object tables do not match expected patches"


tableList = []
for patchno, patchpath in zip(ot_patches, objectTables):
    merian = consolidateMerianCats([patchno], tract)[keepColumns_merian]
    gaapTable = Table.read(patchpath)[keepColumns_gaap]   

    assert np.all(merian['objectId'] == gaapTable['objectId']) , "GAaP table does not match Merian table!"
    gaapTable.remove_columns(["objectId", "coord_ra", "coord_dec", "ebv"])
    tableList.append(hstack([merian, gaapTable]))
    # tableList.append(gaapTable)
fullTable = vstack(tableList)



In [146]:
len(fullTable.colnames), len(fullTable)

(69, 2185323)

In [141]:
# for c in fullTable.colnames:
#     try:
#         if len(fullTable[~fullTable[c].mask])==0:
#             print(c) 
#     except:
#         pass

In [13]:
# outCatFile = os.path.join(catDir, f'objectTable_{tract}_S20A.h5')
# cat = fullTable.to_pandas()
# cat.to_hdf(outCatFile, key='df', mode='w')

: 

: 

In [149]:
outCatFile = os.path.join(catDir, f'objectTable_{tract}_S20A.fits')
print(f"WRITING TO {outCatFile}")
fullTable.write(outCatFile, overwrite=True)
print("SUCCESS")


WRITING TO /scratch/gpfs/am2907/Merian/gaap/S20A/gaapTable/9326/objectTable_9326_S20A.fits
SUCCESS


In [150]:
repo = "/scratch/gpfs/am2907/Merian/gaap/"
catDir = os.path.join(repo, f"S20A/gaapTable/{tract}/")
outCatFile = os.path.join(catDir, f'objectTable_{tract}_S20A.fits')
testing = Table.read(outCatFile)

In [151]:
len(testing.colnames), len(testing)

(69, 2185323)